In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("token")

!git clone https://{token}@github.com/Benedart/RecSys-2022-Challenge-Polimi.git

Cloning into 'RecSys-2022-Challenge-Polimi'...
remote: Enumerating objects: 635, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 635 (delta 9), reused 15 (delta 5), pack-reused 615
Receiving objects: 100% (635/635), 1.06 GiB | 56.70 MiB/s, done.
Resolving deltas: 100% (275/275), done.
Updating files: 100% (205/205), done.


In [2]:
import pandas as pd

path = "/kaggle/input/recommender-system-2022-challenge-polimi/interactions_and_impressions.csv"
Interactions_and_Impressions = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserId', 'ItemId', 'ImpressionList', 'Data'])


content_type = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_type.csv")
content_length = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_length.csv")

ICM_all = pd.concat([content_type, content_length]).sort_values(by='item_id')

Interactions_and_Impressions

,UserId,ItemId,ImpressionList,Data
0,0,21,NaN,0
1,0,21,NaN,0
2,0,21,"20,21,22,23,24,25,26,27,28,29",0
3,0,21,NaN,1
4,0,21,NaN,1
...,...,...,...,...
5826500,41628,20448,NaN,0
5826501,41628,20896,NaN,1
5826502,41628,21506,NaN,1
5826503,41628,22882,NaN,0


In [3]:
Distinct_Interactions = Interactions_and_Impressions.copy(deep=True)
Distinct_Interactions = Distinct_Interactions.drop(["ImpressionList", "Data"], axis=1)
Distinct_Interactions = Distinct_Interactions.drop_duplicates()
Distinct_Interactions

,UserId,ItemId
0,0,21
12,0,22
27,0,24
28,0,44
29,0,54
...,...,...
5826500,41628,20448
5826501,41628,20896
5826502,41628,21506
5826503,41628,22882


In [4]:
mapped_id, original_id = pd.factorize(Distinct_Interactions["UserId"].unique())
user_original_Id_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(Distinct_Interactions["ItemId"].unique())
item_original_Id_to_index = pd.Series(mapped_id, index=original_id)

In [5]:
Distinct_Interactions["UserId"] = Distinct_Interactions["UserId"].map(user_original_Id_to_index)
Distinct_Interactions["ItemId"] = Distinct_Interactions["ItemId"].map(item_original_Id_to_index)
Distinct_Interactions

,UserId,ItemId
0,0,0
12,0,1
27,0,2
28,0,3
29,0,4
...,...,...
5826500,41628,3699
5826501,41628,12693
5826502,41628,15131
5826503,41628,18127


In [6]:
Distinct_Interactions["Rating"]=1
Distinct_Interactions

,UserId,ItemId,Rating
0,0,0,1
12,0,1,1
27,0,2,1
28,0,3,1
29,0,4,1
...,...,...,...
5826500,41628,3699,1
5826501,41628,12693,1
5826502,41628,15131,1
5826503,41628,18127,1


In [7]:
userId_unique = Distinct_Interactions["UserId"].unique()
itemId_unique = Distinct_Interactions["ItemId"].unique()

In [8]:
from sklearn.model_selection import train_test_split
import numpy as np

seed = 42


(user_ids_training_validation, user_ids_test,
 item_ids_training_validation, item_ids_test,
 ratings_training_validation, ratings_test) = train_test_split(Distinct_Interactions.UserId,
                                                               Distinct_Interactions.ItemId,
                                                               Distinct_Interactions.Rating,
                                                               test_size=0.15,
                                                               shuffle=True,
                                                               random_state=seed)

(user_ids_training, user_ids_validation,
 item_ids_training, item_ids_validation,
 ratings_training, ratings_validation) = train_test_split(user_ids_training_validation,
                                                          item_ids_training_validation,
                                                          ratings_training_validation,
                                                          test_size=0.15,
                                                          shuffle=True,
                                                          random_state=seed)

In [9]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [10]:
import scipy.sparse as sps

urm_all = sps.csr_matrix((Distinct_Interactions.Rating, (Distinct_Interactions.UserId, Distinct_Interactions.ItemId)),
                         shape = (num_users, num_items))

urm_train = sps.csr_matrix((ratings_training, (user_ids_training, item_ids_training)),
                           shape = (num_users, num_items))

urm_validation = sps.csr_matrix((ratings_validation, (user_ids_validation, item_ids_validation)),
                                shape = (num_users, num_items))

urm_train_validation = sps.csr_matrix((ratings_training_validation, (user_ids_training_validation, item_ids_training_validation)),
                                      shape = (num_users, num_items))

urm_test = sps.csr_matrix((ratings_test, (user_ids_test, item_ids_test)),
                          shape = (num_users, num_items))

urm_all, urm_train, urm_test, urm_validation, urm_train_validation

(<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 1554639 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 1123226 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 233196 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 198217 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 1321443 stored elements in Compressed Sparse Row format>)

In [11]:
import os
os.chdir( "./RecSys-2022-Challenge-Polimi")

In [12]:
from Recommenders.Recommender_import_list import *
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 1549 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


## Insert model here

In [13]:
model = SLIMElasticNetRecommender

In [14]:
import optuna as op

def objective(trial):
    topK = trial.suggest_int("topK", 3e3, 1e4)
    l1 = trial.suggest_float("l1_ratio", 0.01, 0.1)
    alpha = trial.suggest_float("alpha", 5e-4, 0.005)

    
    recommender = model(urm_train)
    recommender.fit(**trial.params)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [15]:
best_params = {
    "topK": 6468,
    "l1_ratio": 0.03534306561986866,
    "alpha": 0.001,
}

study = op.create_study(direction="maximize")
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=30)

[I 2023-01-13 23:32:30,511] A new study created in memory with name: no-name-8fa881b2-5701-4ae0-b54a-8aa26b4a9f8c


SLIMElasticNetRecommender: Processed 4169 (17.0%) in 5.00 min. Items per second: 13.89
SLIMElasticNetRecommender: Processed 8734 (35.6%) in 10.00 min. Items per second: 14.55
SLIMElasticNetRecommender: Processed 13477 (55.0%) in 15.00 min. Items per second: 14.97
SLIMElasticNetRecommender: Processed 18354 (74.9%) in 20.00 min. Items per second: 15.29
SLIMElasticNetRecommender: Processed 23414 (95.5%) in 25.00 min. Items per second: 15.61
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 26.04 min. Items per second: 15.69
EvaluatorHoldout: Processed 40080 (100.0%) in 44.66 sec. Users per second: 897


[I 2023-01-13 23:59:17,435] Trial 0 finished with value: 0.019956682864429493 and parameters: {'topK': 6468, 'l1_ratio': 0.03534306561986866, 'alpha': 0.001}. Best is trial 0 with value: 0.019956682864429493.


SLIMElasticNetRecommender: Processed 8440 (34.4%) in 5.00 min. Items per second: 28.13
SLIMElasticNetRecommender: Processed 22010 (89.8%) in 10.00 min. Items per second: 36.68
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 10.70 min. Items per second: 38.16
EvaluatorHoldout: Processed 40080 (100.0%) in 32.63 sec. Users per second: 1228


[I 2023-01-14 00:10:32,371] Trial 1 finished with value: 0.019284805389221196 and parameters: {'topK': 6939, 'l1_ratio': 0.07087257428388977, 'alpha': 0.0021548096392657694}. Best is trial 0 with value: 0.019956682864429493.


...

[I 2023-01-14 09:47:26,566] Trial 27 finished with value: 0.019846674706142854 and parameters: {'topK': 4275, 'l1_ratio': 0.03112977199308151, 'alpha': 0.0016519737307700163}. Best is trial 21 with value: 0.01997237172480393.


SLIMElasticNetRecommender: Processed 9439 (38.5%) in 5.00 min. Items per second: 31.46
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 9.74 min. Items per second: 41.94
EvaluatorHoldout: Processed 40080 (100.0%) in 31.75 sec. Users per second: 1262


[I 2023-01-14 09:57:42,742] Trial 28 finished with value: 0.018683284225200103 and parameters: {'topK': 6245, 'l1_ratio': 0.042729381898891895, 'alpha': 0.0049852079252161174}. Best is trial 21 with value: 0.01997237172480393.


SLIMElasticNetRecommender: Processed 6341 (25.9%) in 5.00 min. Items per second: 21.13
SLIMElasticNetRecommender: Processed 15162 (61.9%) in 10.00 min. Items per second: 25.26
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 14.15 min. Items per second: 28.86
EvaluatorHoldout: Processed 40080 (100.0%) in 36.17 sec. Users per second: 1108


[I 2023-01-14 10:12:28,262] Trial 29 finished with value: 0.0194818992966444 and parameters: {'topK': 5249, 'l1_ratio': 0.05063548223726836, 'alpha': 0.002212234479041836}. Best is trial 21 with value: 0.01997237172480393.


In [16]:
study.best_params

{'topK': 6117,
 'l1_ratio': 0.018920292136425064,
 'alpha': 0.0012529892110969758}

In [17]:
final = model(urm_train_validation)
final.fit(**study.best_params)

SLIMElasticNetRecommender: Processed 3134 (12.8%) in 5.00 min. Items per second: 10.44
SLIMElasticNetRecommender: Processed 6508 (26.6%) in 10.00 min. Items per second: 10.84
SLIMElasticNetRecommender: Processed 10002 (40.8%) in 15.00 min. Items per second: 11.11
SLIMElasticNetRecommender: Processed 13558 (55.3%) in 20.00 min. Items per second: 11.30
SLIMElasticNetRecommender: Processed 17173 (70.1%) in 25.00 min. Items per second: 11.45
SLIMElasticNetRecommender: Processed 20803 (84.9%) in 30.01 min. Items per second: 11.55
SLIMElasticNetRecommender: Processed 24500 (100.0%) in 35.01 min. Items per second: 11.66
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 35.02 min. Items per second: 11.66


In [18]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 931 ( 2.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 40698 (100.0%) in 51.99 sec. Users per second: 783


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.056172                 0.093717  0.087615  0.027672    0.045144   
 
             MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.18309  0.089816  0.068456  0.366922      0.223347  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.977636          0.358716    0.977636       0.021583   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            8.937814                   0.993193             0.042442   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                      

In [19]:
final.save_model(".", "SLIM_Elastic_train_validation.zip")

!cp SLIM_Elastic_train_validation.zip ..
os.chdir("..")
!rm -r RecSys-2022-Challenge-Polimi

SLIMElasticNetRecommender: Saving model in file '.SLIM_Elastic_train_validation.zip'
SLIMElasticNetRecommender: Saving complete
